<a href="https://colab.research.google.com/github/carlosegn/proyecciones-ventas-alimenticias/blob/main/Proyeccion_ventas_alimenticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CODING DOJO: Proyecto de proyección de ventas alimenticias**
## Creado por: Carlos Guerrero

* Fecha creado: 16/02/2023
* Fecha actualización: 24/02/2023
* Descripción: Proyección de ventas para productos alimenticios vendidos en diversas tiendas.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
archivo = '/content/drive/MyDrive/CodingDojo/sales_predictions.csv'
df = pd.read_csv(archivo)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## Exploración y limpieza de los datos

In [3]:
# Filas y columas que tiene el dataframe
df.shape

(8523, 12)

In [4]:
# Información de cada variable (columna)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [7]:
# Limpieza de duplicados
df.duplicated().sum()

0

No existen valores duplicados

In [8]:
# Identificar valores faltantes
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

Se observan varios registros faltantes en las variables de peso del producto (Item_Weight) y tamaño de la tienda (Outlet_Size)

### Análisis de variable Outlet_Size

In [9]:
# Valores de la variable Outlet_Size
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [10]:
# Valores de la variable Outlet_Type
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [11]:
# Valores de la variable Outlet_Location_Type
df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [12]:
# Relación entre Outlet_Size, Outlet_Type y Outlet_Location_Type
df.groupby(['Outlet_Size', 'Outlet_Type', 'Outlet_Location_Type'])['Outlet_Type'].count()

Outlet_Size  Outlet_Type        Outlet_Location_Type
High         Supermarket Type1  Tier 3                  932
Medium       Supermarket Type1  Tier 1                  930
             Supermarket Type2  Tier 3                  928
             Supermarket Type3  Tier 3                  935
Small        Grocery Store      Tier 1                  528
             Supermarket Type1  Tier 1                  930
                                Tier 2                  930
Name: Outlet_Type, dtype: int64

In [13]:
# Relación entre Outlet_Type y Outlet_Location_Type
df.groupby(['Outlet_Type', 'Outlet_Location_Type'])['Outlet_Type'].count()

Outlet_Type        Outlet_Location_Type
Grocery Store      Tier 1                   528
                   Tier 3                   555
Supermarket Type1  Tier 1                  1860
                   Tier 2                  2785
                   Tier 3                   932
Supermarket Type2  Tier 3                   928
Supermarket Type3  Tier 3                   935
Name: Outlet_Type, dtype: int64

In [14]:
# Relación de los valores nulos en Outlet_Size con Outlet_Type y Outlet_Location_Type
filtro_outlet_size = df['Outlet_Size'].isna()
df[filtro_outlet_size].groupby(['Outlet_Type', 'Outlet_Location_Type'])['Outlet_Type'].count()

Outlet_Type        Outlet_Location_Type
Grocery Store      Tier 3                   555
Supermarket Type1  Tier 2                  1855
Name: Outlet_Type, dtype: int64

**Análisis**  
De acuerdo al análisis realizado puedo encontrar una relación entre el tipo de Outlet (Outlet_Type), la ubicación del Outlet (Outlet_Location_Type) y el tamaño del Outlet (Outlet_Size).

* Todos los Outlet de tamaño grande (High) son Supermarket Type1 y Tier 3
* Los outlet medianos que son de tipo Supermarket Type1 sólo están en Tier 1
* Los outlet medianos de tipo Supermarket Type2 y Type3 sólo están en Tier 3
* Todos los outlet de tipo Grocery Store son de tamaño pequeño (Small)
* Todos los outlet de tipo Supermarket Type1 que están en Tier 1 y 2, son de tamaño pequeño (Small)

Los valores nulos en la variable Outlet_Size corresponden a:  
* 555 valores Grocery Store, por lo tanto se podría imputar el valor "Small"
* 1855 valores Supermarket Type1 de Tier 2, por lo tanto se podría imputar el valor "Small"

In [15]:
# Imputar todos los valores nulos en Outlet_Size con "Small" y guardar en un nuevo dataframe
df_final = df
df_final['Outlet_Size'].fillna('Small', inplace = True)
df_final.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

### Análisis de variable Item_Weight

In [16]:
# Valores de la variable Item_Weight
df_final['Item_Weight'].value_counts()

12.150    86
17.600    82
13.650    77
11.800    76
15.100    68
          ..
7.275      2
7.685      1
9.420      1
6.520      1
5.400      1
Name: Item_Weight, Length: 415, dtype: int64

In [17]:
# Valores de la variable Item_Identifier
df_final['Item_Identifier'].value_counts()

FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64

In [18]:
# Observar valores únicos en cada variable
df_final.nunique()

Item_Identifier              1559
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

In [19]:
# Relación entre Item_Weight e Item_Identifier
df_final.groupby(['Item_Identifier'])['Item_Weight'].count()

Item_Identifier
DRA12    6
DRA24    5
DRA59    6
DRB01    2
DRB13    5
        ..
NCZ30    6
NCZ41    5
NCZ42    5
NCZ53    4
NCZ54    5
Name: Item_Weight, Length: 1559, dtype: int64

**Análisis**  
De acuerdo a los datos, todos los Item_Identifier se corresponden con un Item_Weight, por lo que podríamos imputar los valores faltantes en Item_Weight consultando los valores de Item_Identifier e imputando un valor conocido que se obtenga de otro registro

In [20]:
# Obtener los Item_Identifier con valores de Item_Weight NaN
filtro_item_weight = df_final['Item_Weight'].isna()
df_final[filtro_item_weight]['Item_Identifier']

7       FDP10
18      DRI11
21      FDW12
23      FDC37
29      FDC14
        ...  
8485    DRK37
8487    DRG13
8488    NCN14
8490    FDU44
8504    NCN18
Name: Item_Identifier, Length: 1463, dtype: object

In [21]:
# El filtro nos da un falor de tipo de series de Panda, por lo que podemos iterar en los items de la serie
type(df_final[filtro_item_weight]['Item_Identifier'])

pandas.core.series.Series

In [22]:
# Iteramos por la serie de valores que tienen un valor NaN en Item_Weight
for index, value in df_final[filtro_item_weight]['Item_Identifier'].iteritems():
  
  # Creamos un filtro para cada valor de Item_Identifier
  filtro_id = df_final['Item_Identifier'] == value

  # Guardamos el peso máximo para ese Item_Identifier, funciona con mean() o min() también ya que son valores únicos
  peso = df_final[filtro_id]['Item_Weight'].max()

  # Cambiamos el valor en el indice obtenido de la serie y en el atributo Item_Weight
  df_final.at[index, 'Item_Weight'] = peso

In [24]:
# Confirmamos si quedan valores nulos
df_final.isna().sum()

Item_Identifier              0
Item_Weight                  4
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

Todavía quedan 4 registros con Item_Weight NaN, esto debido a que no existía ningún otro registro con un Item_Identifier igual, por lo que voy a eliminar estos registros ya que representan menos del 0.05% de los datos

In [25]:
df_final.dropna(inplace=True)
df_final.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

La base de datos ya no tiene valores faltantes.

### Encontrar y resolver categorías inconsistentes

In [33]:
df_final.head(1)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.3,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138


In [31]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8519 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8519 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8519 non-null   object 
 3   Item_Visibility            8519 non-null   float64
 4   Item_Type                  8519 non-null   object 
 5   Item_MRP                   8519 non-null   float64
 6   Outlet_Identifier          8519 non-null   object 
 7   Outlet_Establishment_Year  8519 non-null   int64  
 8   Outlet_Size                8519 non-null   object 
 9   Outlet_Location_Type       8519 non-null   object 
 10  Outlet_Type                8519 non-null   object 
 11  Item_Outlet_Sales          8519 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.2+ KB


In [34]:
# Revisar categorías en valores Item_Fat_Content, Item_Type, Outlet_Identifier
# El resto de columnas ya se revisaron anteriormente sin encontrar inconsistencias
df_final['Item_Fat_Content'].value_counts()

Low Fat    5088
Regular    2886
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [35]:
df_final['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1199
Household                 910
Frozen Foods              855
Dairy                     681
Canned                    649
Baking Goods              647
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [36]:
df_final['Outlet_Identifier'].value_counts()

OUT013    932
OUT027    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    527
Name: Outlet_Identifier, dtype: int64

Las columnas Item_Type y Outlet_Identifier parecen tener información correcta, la columna Item_Fat_Content tiene categorías que se pueden corregir.

In [37]:
# Corregir categorías inconsistentes en Item_Fat_Content
df_final['Item_Fat_Content'].replace('LF', 'Low Fat', inplace = True)
df_final['Item_Fat_Content'].replace('low fat', 'Low Fat', inplace = True)
df_final['Item_Fat_Content'].replace('reg', 'Regular', inplace = True)
df_final['Item_Fat_Content'].value_counts()

Low Fat    5516
Regular    3003
Name: Item_Fat_Content, dtype: int64

### Valores sumarizados para columnas numéricas

In [48]:
df_final.describe().loc[['min', 'max', 'mean']]

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
min,4.55500,0.000000,31.290000,1985.000000,33.290000
max,21.35000,0.328391,266.888400,2009.000000,13086.964800
mean,12.87542,0.066112,141.010019,1997.837892,2181.188779


Se muestran los valores minimos, máximos y media de todas las columnas numéricas.